# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# ML classifiers
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# load data from database
def load_data(db_name, table_name='Message', n_sample=0):
    '''
    Loads data from the SQLite database.
    
    Input:
        db_name: database name
        table_name: table name with messages data
        n_sample: if not 0 a sampled dataset is returned with n_sample rows 
    
    '''
    
    print('Loading data...')
    
    engine = create_engine('sqlite:///' + db_name)
    df = pd.read_sql_table(table_name, engine)
    
    # apply sampling
    if (n_sample != 0):
        df = df.sample(n=n_sample, axis=0, random_state=0)
 
    X = df['message'].values 
    y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
    
    print('Data loaded.')
    
    return X, y


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    '''
    Normalizes, lemmatizes, and tokenizes text.
    '''
    # normalization
    text = re.sub(r'[^a-z0-9]', ' ', text.lower())
    
    # tokenization
    words = word_tokenize(text)

    # lemmatizing
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    # Stop Word removal
    words = [w for w in words if w not in stopwords.words('english')]
    
    return words


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def build_model():
    '''
    Builds the 1st model with RandomForest classifier
    and default hyper parameters.
    '''
    model = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier(random_state=0))
    ])
    
    print('Model created.')
    
    return model


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
def train(X, y, model):
    '''
    Trains the model.
    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    
    print('Training the model...')
    
    model.fit(X_train, y_train)
    
    print('Model is trained.')
    print('Predicting the model...')
    
    y_pred = model.predict(X_test)
    
    print('Model is done.')
    
    return y_test, y_pred


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def display_results(y_test, y_pred):
    '''
    Displays the results using accuracy, F1 score, precision and recall
    for each output category.
    '''
    for i, col in enumerate(y_test.columns.values): 
        print(f'Category: {col}')
        y_true = y_test.iloc[:,i]
        y_label = y_pred[:,i]
        accuracy = (y_true == y_label).mean()
        
        print('Accuracy: {:.4f}'.format(accuracy))
        print('Classification report:')
        print(classification_report(y_true, y_label))
        print('--------------------------------------------------------')
        print()
        
    # total accuracy
    total_accuracy = (y_pred == y_test).mean().sum()/len(y_test.columns)  
        
    print('Average accuracy: {:.4f}'.format(total_accuracy))
    

### Model-1 (RandomForestClassifier)

Average accuracy: 0.9411

In [11]:
# Train model-1
X, y = load_data('DisasterResponse.db')
model = build_model()
y_test, y_pred = train(X, y, model)
display_results(y_test, y_pred)

Loading data...
Data loaded.
Model created.
Training the model...
Model is trained.
Predicting the model...
Model is done.
Category: related
Accuracy: 0.8114
Classification report:
             precision    recall  f1-score   support

          0       0.62      0.51      0.56      1509
          1       0.86      0.91      0.88      5004
          2       0.35      0.49      0.41        41

avg / total       0.80      0.81      0.81      6554

--------------------------------------------------------

Category: request
Accuracy: 0.8853
Classification report:
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5419
          1       0.82      0.44      0.57      1135

avg / total       0.88      0.89      0.87      6554

--------------------------------------------------------

Category: offer
Accuracy: 0.9965
Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### Model-2 (RandomForestClassifier with MultiOutputClassifier, tuned parameters)

Average accuracy: 0.9485

In [44]:
def build_model_2():  
    '''
    Builds the 2st model using GridSearchCV for tuning the hyper parameters.
    Note: We will not tune all parameters at once due to the performance issues.
          We'll tune step by step taking a single parameter with its testing 
          arguments and then we'll repeat the tuning until all parameters are checked.
          To speed up the tuning we'll also reduce the size of the dataset using 
          a sample of 2.500 rows.
    '''    
    model = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(
            random_state = 0,
            n_estimators = 100,
            max_features = 'auto',
            min_samples_leaf = 1,
            min_samples_split = 2,
            bootstrap = True
        )))
    ])
    
    params = {
        'clf__estimator__n_estimators': [50, 100, 250],    # best: 100
        'clf__estimator__min_samples_leaf': [1, 2],        # best: 1 
        'clf__estimator__min_samples_split': [2, 3, 5],    # best: 2 
        'clf__estimator__criterion': ['entropy', 'gini'],  # best: gini
        'clf__estimator__max_features': ['auto', 'log2'],  # best: auto 
        'clf__estimator__bootstrap': [True, False],        # best: True
        'vect__ngram_range': [(1,1), (1,2)],               # best: (1,2)
        'vect__max_df': [1.0, 0.95],                       # best: 1.0
        'vect__min_df': [1, 2]                             # best: 2
    }

    cv = GridSearchCV(model, param_grid=params, verbose=3, n_jobs=-1)
    
    return cv


In [44]:
# Train model-2 using GridSearchCV

X, y = load_data('DisasterResponse.db', 'Message', 2500)
model = build_model_2()
y_test, y_pred = train(X, y, model)
display_results(y_test, y_pred)

Loading data...
Data loaded.
Training the model...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 1) ......
[CV]  vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 1), score=0.18, total=  11.2s
[CV] vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 1) ......


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   14.1s remaining:    0.0s


[CV]  vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 1), score=0.188, total=  11.0s
[CV] vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 1) ......


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   28.1s remaining:    0.0s


[CV]  vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 1), score=0.164, total=  11.1s
[CV] vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 2) ......
[CV]  vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 2), score=0.196, total=  11.8s
[CV] vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 2) ......
[CV]  vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 2), score=0.196, total=  11.8s
[CV] vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 2) ......
[CV]  vect__max_df=1.0, vect__min_df=1, vect__ngram_range=(1, 2), score=0.192, total=  11.8s
[CV] vect__max_df=1.0, vect__min_df=2, vect__ngram_range=(1, 1) ......
[CV]  vect__max_df=1.0, vect__min_df=2, vect__ngram_range=(1, 1), score=0.216, total=  10.7s
[CV] vect__max_df=1.0, vect__min_df=2, vect__ngram_range=(1, 1) ......
[CV]  vect__max_df=1.0, vect__min_df=2, vect__ngram_range=(1, 1), score=0.212, total=  10.8s
[CV] vect__max_df=1.0, vect__min_df=2, vect__ngram_range=(1, 1) ......
[CV]  vect__max_

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  5.6min finished


Model is trained.
Predicting the model...
Model is done.
Category: related
Accuracy: 0.7800
Classification report:
             precision    recall  f1-score   support

          0       0.70      0.28      0.40        58
          1       0.82      0.93      0.88       192
          2       0.00      0.00      0.00         0

avg / total       0.79      0.78      0.76       250

--------------------------------------------------------

Category: request
Accuracy: 0.8840
Classification report:
             precision    recall  f1-score   support

          0       0.88      1.00      0.93       207
          1       0.94      0.35      0.51        43

avg / total       0.89      0.88      0.86       250

--------------------------------------------------------

Category: offer
Accuracy: 0.9960
Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       249
          1       0.00      0.00      0.00         1

avg / total

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [45]:
# An example of CountVectorizer parameters (that were tuned together in a single process)

model.best_params_

{'vect__max_df': 1.0, 'vect__min_df': 2, 'vect__ngram_range': (1, 2)}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Model-3 (AdaBoostClassifier)

Average accuracy: 0.9468

In [3]:
def build_model_AdaBoost():
    '''
    Builds the model with AdaBoost classifier with default parameters.
    '''
    model = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=0)))
    ])
    
    print('Model created.')
    
    return model

In [11]:
# Train model AdaBoost

X, y = load_data('DisasterResponse.db')
model = build_model_AdaBoost()
y_test, y_pred = train(X, y, model)
display_results(y_test, y_pred)

Loading data...
Data loaded.
Model created.
Training the model...
Model is trained.
Predicting the model...
Model is done.
Category: related
Accuracy: 0.7757
Classification report:
             precision    recall  f1-score   support

          0       0.61      0.15      0.23      1509
          1       0.79      0.97      0.87      5004
          2       0.43      0.15      0.22        41

avg / total       0.74      0.78      0.72      6554

--------------------------------------------------------

Category: request
Accuracy: 0.8885
Classification report:
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5419
          1       0.77      0.51      0.61      1135

avg / total       0.88      0.89      0.88      6554

--------------------------------------------------------

Category: offer
Accuracy: 0.9954
Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531

### Model-4 (XGBoost)

Average accuracy: 0.9468

In [15]:
!pip install xgboost

    100% |████████████████████████████████| 142.8MB 249kB/s eta 0:00:01 1% |▋                               | 2.8MB 31.7MB/s eta 0:00:05    2% |█                               | 4.2MB 25.9MB/s eta 0:00:06    7% |██▍                             | 10.7MB 28.3MB/s eta 0:00:05    9% |███                             | 13.5MB 33.3MB/s eta 0:00:04    12% |████                            | 17.7MB 34.3MB/s eta 0:00:04    14% |████▌                           | 20.2MB 31.6MB/s eta 0:00:04    18% |██████                          | 26.6MB 20.0MB/s eta 0:00:06    19% |██████▎                         | 27.9MB 31.6MB/s eta 0:00:04    21% |██████▉                         | 30.4MB 26.3MB/s eta 0:00:05    31% |██████████                      | 44.9MB 23.7MB/s eta 0:00:05    32% |██████████▍                     | 46.2MB 29.7MB/s eta 0:00:04    33% |██████████▋                     | 47.3MB 25.1MB/s eta 0:00:04    35% |███████████▍                    | 50.8MB 17.7MB/s eta 0:00:06    42% |█████████████▌     

In [16]:
from xgboost import XGBClassifier

In [10]:
# Previously tuned by GridSearchCV that has given the following best params
# for XGBoost classifier:

model.best_params_

{'clf__estimator__learning_rate': 0.1,
 'clf__estimator__max_depth': 3,
 'clf__estimator__n_estimators': 300}

In [17]:
def build_model_XGBoost():
    '''
    Builds the model with XGBoost classifier.
    '''    
    model = Pipeline([
        ('features', FeatureUnion([
            ('text', Pipeline([
                ('vect', CountVectorizer(tokenizer = tokenize)),
                ('tfidf', TfidfTransformer()),
                ('svd', TruncatedSVD(algorithm='randomized', n_components=100)), #for XGB
            ]))
        ])),
        ('clf', MultiOutputClassifier(
            XGBClassifier(
                max_depth=3, 
                n_estimators=300, 
                learning_rate=0.1, 
                random_state=0))),
    ])
    
    print('Model created.')
    
    return model


In [18]:
# Train model XGBoost

X, y = load_data('DisasterResponse.db')
model = build_model_XGBoost()
y_test, y_pred = train(X, y, model)
display_results(y_test, y_pred)

Loading data...
Data loaded.
Model created.
Training the model...
Model is trained.
Predicting the model...
Model is done.
Category: related
Accuracy: 0.8131
Classification report:
             precision    recall  f1-score   support

          0       0.67      0.41      0.51      1509
          1       0.84      0.94      0.89      5004
          2       0.72      0.32      0.44        41

avg / total       0.80      0.81      0.80      6554

--------------------------------------------------------

Category: request
Accuracy: 0.8943
Classification report:
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5419
          1       0.78      0.54      0.64      1135

avg / total       0.89      0.89      0.89      6554

--------------------------------------------------------

Category: offer
Accuracy: 0.9965
Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Winning model: model-2

In [48]:
def build_winning_model():
    '''
    Uses the winning model with the best hyper parameters found by GridSearchCV.
    '''        
    model = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1,2), max_df=1.0, min_df=2)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(
            random_state = 0,
            n_estimators = 100,
            max_features = 'auto',
            min_samples_leaf = 1,
            min_samples_split = 2,
            bootstrap = True
        )))
    ])
    
    print('Model created.')
    
    return model


### 9. Export your model as a pickle file

In [1]:
def save_model(model, model_filepath):
    '''
    Serializes model and saves it as a file.
    
    Input:
        model: model to save
        model_filepath: path of a model file
    '''
    with open(model_filepath, 'wb') as file:
        pickle.dump(model, file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.